# RunModel with Heterogeneous Input: Python model execution

The RunModel class is capable of passing input in different formats into a single computational model. This means that the samples passed into a model can be passed as:
- floating point values
- numpy arrays
- lists 
- tuples
- lists of other iterables
- numpy arrays of other iterables
- or any combination of the above

In the examples below, we demonstrate the use of a Python computational model with inputs that are combinations of the above.

Some notes on their use:
1. UQpy converts all sample input to a numpy array with at least two dimensions. The first dimension, i.e. len(samples) must correspond to the number of samples being passed for model execution. The second dimension, i.e. len(samples[0]) must correspond to the number of variables that each sample possesses.
2. Each individual sample, i.e. sample[j], may be composed of multiple data types -- with each variable having a different data type. For example, sample[j][k] may be a floating point value and sample[j][l] may be an array of arbitrary dimension.
3. If a specific variable has multiple dimensions, the user may specify the index to be return in the input file. For example, the place holder for a variable x1 corresponding to sample[j][l] that is an array of shape (1,4) can be read as <x1[0, 3]>, which will return the final (0,3) component of samples[j][l].
4. If the user does not specify the index for a multidimensional variable, then the entire multidimensional variable is flattened and written with comma delimiters.

Michael D. Shields  
29 April 2020

### Python Model Summary
Examples 1-2:
The provided Matlab models take the sum of three random variables:  
$s = \sum_{i=1}^3 x_i$  
$x_i \sim N(0,1)$

Example 3:
The provided Matlab model takes the product of a random variable and the determinant of a random matrix:  
$z = x \det(Y)$  
$x \sim N(0,1)$  
$y$ is a 3x3 matrix of standard normal random variables.

The Python model may be provided as either a class or a function. The examples below explore both cases.

In [1]:
from UQpy.SampleMethods import MCS
from UQpy.RunModel import RunModel
import matplotlib.pyplot as plt
import time
import numpy as np

### Pick which model to run
Options:
- 'all'
- 'scalar'
- 'vector'
- 'mixed'
- 'fixed'

In [2]:
pick_model = 'all'

## Example 1: Three scalar random variables
In this example, we pass three scalar random variables. Note that this is different from assigning a single variable with three components, which will be handled in the following example. 

Here we will pass the samples both as an ndarray and as a list. Recall that UQpy converts all samples into an ndarray of at least two dimensions internally.

In [3]:
if pick_model == 'scalar' or pick_model =='vector' or pick_model == 'all':
    # Call MCS to generate samples
    # THIS WILL NEED TO BE REWRITTEN WITH DISTRIBUTION AND MCS UPDATES --------------------------------------------
    x_mcs = MCS(dist_name=['Normal','Normal','Normal'], dist_params=[[0,1],[0,1],[0,1]], nsamples=5, 
                var_names = ['var1', 'var11', 'var111']) 
    # -------------------------------------------------------------------------------------------------------------

    # UQpy returns samples as an ndarray. Convert them to a list for part 1.2
    x_mcs_list = list(x_mcs.samples)
    print("Monte Carlo samples of three random variables from a standard normal distribution.")
    print('Samples stored as an array:')
    print('Data type:', type(x_mcs.samples))
    print('Number of samples:', len(x_mcs.samples))
    print('Dimensions of samples:', np.shape(x_mcs.samples))
    print('Samples')
    print(x_mcs.samples)
    print()
    print('Samples stored as a list:')
    print('Data type:', type(x_mcs_list))   
    print('Number of samples:', len(x_mcs_list))
    print('Dimensions of samples:', np.shape(x_mcs_list))
    print('Samples:')
    print(x_mcs_list)

Monte Carlo samples of three random variables from a standard normal distribution.
Samples stored as an array:
Data type: <class 'numpy.ndarray'>
Number of samples: 5
Dimensions of samples: (5, 3)
Samples
[[-0.0397043   0.51667765  0.5388383 ]
 [-0.64878299  0.34359211  0.84027172]
 [ 0.61856398  0.9772204   0.39248505]
 [-0.24540884  1.62960553  0.2364994 ]
 [ 1.23323326 -1.72328346 -0.527537  ]]

Samples stored as a list:
Data type: <class 'list'>
Number of samples: 5
Dimensions of samples: (5, 3)
Samples:
[array([-0.0397043 ,  0.51667765,  0.5388383 ]), array([-0.64878299,  0.34359211,  0.84027172]), array([0.61856398, 0.9772204 , 0.39248505]), array([-0.24540884,  1.62960553,  0.2364994 ]), array([ 1.23323326, -1.72328346, -0.527537  ])]


### 1.1 Pass samples as ndarray, Python class called, serial execution  
This examples uses the following files:
- model_script = python_model.py

In [6]:
if pick_model == 'scalar' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m11 = RunModel(ntasks=1, model_script='python_model.py', model_object_name='SumRVs', model_dir='Python_Runs', verbose=True)
    m11.run(samples=x_mcs.samples,)
    t_ser_python = time.time() - t
    print("\nTime for serial execution:")
    print(t_ser_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m11.qoi_list)


UQpy: The following directory has been created for model evaluations: 
/Users/MichaelShields/Documents/SURG/Code/UQpy/V3/RunModel/Python_Example/Python_Runs_2020_05_12_04_59_722304_PM

UQpy: The model files have been copied to the following directory for evaluation: 
/Users/MichaelShields/Documents/SURG/Code/UQpy/V3/RunModel/Python_Example/Python_Runs_2020_05_12_04_59_722304_PM

UQpy: No samples are provided. Creating the object and building the model directory.


UQpy: All model evaluations will be executed from the following directory: 
/Users/MichaelShields/Documents/SURG/Code/UQpy/V3/RunModel/Python_Example/Python_Runs_2020_05_12_04_59_722304_PM

UQpy: The model class that will be run: SumRVs

UQpy: Performing serial execution of a Python model.


UQpy: Serial execution of the python model complete.


UQpy: Returning to the parent directory:
/Users/MichaelShields/Documents/SURG/Code/UQpy/V3/RunModel/Python_Example

Time for serial execution:
0.018716096878051758

The values return

### 1.2 Pass samples as list, Python function called, parallel execution  
This examples uses the following files:
- model_script = python_model.py

In [7]:
if pick_model == 'scalar' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m12 = RunModel(samples=x_mcs_list, ntasks=2, model_script='python_model.py', 
                 model_object_name='sum_rvs', model_dir='Python_Runs')
    t_par_python = time.time() - t
    print("\nTime for parallel execution:")
    print(t_par_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m12.qoi_list)


Time for parallel execution:
0.0401301383972168

The values returned from the Matlab simulation:
[array([1.01581165]), array([0.53508083]), array([1.98826942]), array([1.62069609]), array([-1.0175872])]


## Example 2: Single tri-variate random variable
In this example, we pass three random variables in as a trivariate random variable. Note that this is different from assigning three scalar random variables, which was be handled in Example 1.

Again, we will pass the samples both as an ndarray and as a list. Recall that UQpy converts all samples into an ndarray of at least two dimensions internally.

### Restructure the samples
To pass the samples in as a single tri-variate variable, we need reshape the samples from shape (5, 3) to shape (5, 1, 3)

In [8]:
if pick_model == 'vector' or pick_model == 'all':
    x_mcs_tri = x_mcs.samples.reshape(5, 1, 3)
    x_mcs_tri_list = list(x_mcs_tri)

    print("Monte Carlo samples of three random variables from a standard normal distribution.")
    print('Samples stored as an array:')
    print('Data type:', type(x_mcs_tri))
    print('Number of samples:', len(x_mcs_tri))
    print('Dimensions of samples:', np.shape(x_mcs_tri))
    print('Samples')
    print(x_mcs_tri)
    print()
    print('Samples stored as a list:')
    print('Data type:', type(x_mcs_tri_list))   
    print('Number of samples:', len(x_mcs_tri_list))
    print('Dimensions of samples:', np.shape(x_mcs_tri_list))
    print('Samples:')
    print(x_mcs_tri_list)

Monte Carlo samples of three random variables from a standard normal distribution.
Samples stored as an array:
Data type: <class 'numpy.ndarray'>
Number of samples: 5
Dimensions of samples: (5, 1, 3)
Samples
[[[-0.0397043   0.51667765  0.5388383 ]]

 [[-0.64878299  0.34359211  0.84027172]]

 [[ 0.61856398  0.9772204   0.39248505]]

 [[-0.24540884  1.62960553  0.2364994 ]]

 [[ 1.23323326 -1.72328346 -0.527537  ]]]

Samples stored as a list:
Data type: <class 'list'>
Number of samples: 5
Dimensions of samples: (5, 1, 3)
Samples:
[array([[-0.0397043 ,  0.51667765,  0.5388383 ]]), array([[-0.64878299,  0.34359211,  0.84027172]]), array([[0.61856398, 0.9772204 , 0.39248505]]), array([[-0.24540884,  1.62960553,  0.2364994 ]]), array([[ 1.23323326, -1.72328346, -0.527537  ]])]


### 2.1 Pass samples as ndarray, Python function called, serial execution
This example uses the following files:
- model_script = python_model.py

In [9]:
if pick_model == 'vector' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m21 = RunModel(samples=x_mcs_tri, ntasks=1, model_script='python_model.py', 
                 model_object_name='sum_rvs_vec', model_dir='Python_Runs')
    t_ser_python = time.time() - t
    print("\nTime for serial execution:")
    print(t_ser_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m21.qoi_list)


Time for serial execution:
0.008770227432250977

The values returned from the Matlab simulation:
[array([1.01581165]), array([0.53508083]), array([1.98826942]), array([1.62069609]), array([-1.0175872])]


### 2.2 Pass samples as list, Python class called, parallel execution
This example uses the following files:
- model_script = python_model.py

In [10]:
if pick_model == 'vector' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m22 = RunModel(samples=x_mcs_tri_list, ntasks=2, model_script='python_model.py', 
                 model_object_name='SumRVs', model_dir='Python_Runs')
    t_par_python = time.time() - t
    print("\nTime for parallel execution:")
    print(t_par_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m22.qoi_list)


Time for parallel execution:
0.027418136596679688

The values returned from the Matlab simulation:
[array([1.01581165]), array([0.53508083]), array([1.98826942]), array([1.62069609]), array([-1.0175872])]


## Example 3: Passing a scalar and an array to RunModel
In this example, we pass a single scalar random variable as well as an array into a Matlab model.

Again, we will pass the samples both as an ndarray and as a list. Recall that UQpy converts all samples into an ndarray of at least two dimensions internally.

In [11]:
if pick_model == 'mixed' or pick_model =='vector' or pick_model == 'all':
    # Call MCS to generate samples
    # THIS WILL NEED TO BE REWRITTEN WITH DISTRIBUTION AND MCS UPDATES --------------------------------------------
    # First generate the scalar random variable
    x_mcs1 = MCS(dist_name=['Normal'], dist_params=[[0,1]], nsamples=5, var_names = ['var1']) 
    # Next generate a 3x3 random matrix
    x_mcs2 = MCS(dist_name=['Normal','Normal','Normal'], dist_params=[[0,1],[0,1],[0,1]], nsamples=15)
    x_mcs_array = x_mcs2.samples.reshape((5,3,3))
    # -------------------------------------------------------------------------------------------------------------
    
    print("Monte Carlo samples of a single random variable from a standard normal distribution.")
    print('Samples stored as an array:')
    print('Data type:', type(x_mcs1.samples))
    print('Number of samples:', len(x_mcs1.samples))
    print('Dimensions of samples:', np.shape(x_mcs1.samples))
    print('Samples')
    print(x_mcs1.samples)
    print()
    print("Monte Carlo samples of a 3x3 matrix of standard normal random variables.")
    print('Samples stored as an array:')
    print('Data type:', type(x_mcs_array))
    print('Number of samples:', len(x_mcs_array))
    print('Dimensions of samples:', np.shape(x_mcs_array))
    print('Samples')
    print(x_mcs_array)
    print()
    
    # Create a set of samples to be passed into RunModel
    # Here we need to create the mixed samples such that each sample has a single scalar and a single 3x3 matrix.
    # This data structure is essential to passing the input to UQpy correctly.
    x_mixed = []
    for i in range(5):
        x_mixed.append([x_mcs1.samples[i], x_mcs_array[i]])
        
    print("Combined samples with a scalar and a 3x3 matrix of standard normal random variables.")
    print('Samples stored as a list:')
    print('Data type:', type(x_mixed))
    print('Number of samples:', len(x_mixed))
    print('Dimensions of samples:', np.shape(x_mixed))
    print('Samples')
    print(x_mixed)
    print()
    
    x_mixed_array = np.atleast_2d(np.asarray(x_mixed))
    print("Combined samples with a scalar and a 3x3 matrix of standard normal random variables.")
    print('Samples stored as ndarray:')
    print('Data type:', type(x_mixed_array))
    print('Number of samples:', len(x_mixed_array))
    print('Dimensions of samples:', np.shape(x_mixed_array))
    print('Samples')
    print(x_mixed_array)
    print()
    
    # Notice that, in both the ndarray case and the list case, the samples have dimension (5,2). That is, there
    # are five samples of two variables. The first variable is a scalar. The second variable is a 3x3 matrix.

Monte Carlo samples of a single random variable from a standard normal distribution.
Samples stored as an array:
Data type: <class 'numpy.ndarray'>
Number of samples: 5
Dimensions of samples: (5, 1)
Samples
[[ 0.00899302]
 [ 0.06658769]
 [-1.22778452]
 [-0.13573769]
 [-0.27684137]]

Monte Carlo samples of a 3x3 matrix of standard normal random variables.
Samples stored as an array:
Data type: <class 'numpy.ndarray'>
Number of samples: 5
Dimensions of samples: (5, 3, 3)
Samples
[[[ 2.1535102   0.1799892  -0.16889527]
  [-0.5214708  -1.22103583 -1.36417229]
  [ 1.95654882  1.58694939 -0.68619824]]

 [[-0.91615269 -0.17857194  0.03673987]
  [ 0.33356414 -0.67773133 -0.55117228]
  [ 0.59363143 -0.08381811 -1.23231183]]

 [[-0.03362657 -0.54695966 -0.00411439]
  [-0.29453249 -0.35960291  0.32253892]
  [ 0.48450073  0.78204757 -2.36459447]]

 [[-0.66210856  2.90532134 -1.56399743]
  [-0.35158539  0.26380749 -0.49864218]
  [-0.13391552 -0.1415161   0.34373841]]

 [[-1.59389912  1.09534476 -1.

### 3.1 Pass samples as ndarray, Python class called, serial execution 
This examples uses the following files:
- model_script = python_model.py

In [12]:
if pick_model == 'mixed' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m31 = RunModel(samples=x_mixed_array, ntasks=1, model_script='python_model.py', 
                 model_object_name='DetRVs', model_dir='Python_Runs', vec=False)
    t_ser_python = time.time() - t
    print("\nTime for serial execution:")
    print(t_ser_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m31.qoi_list)


Time for serial execution:
0.010289907455444336

The values returned from the Matlab simulation:
[array([0.05088167]), array([-0.04821253]), array([-0.33834807]), array([-0.05412348]), array([0.36551101])]


### 3.2 Pass samples as list, Python function called, parallel execution 
This examples uses the following files:
- model_script = python_model.py

In [13]:
if pick_model == 'mixed' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    # Note that the parallel model_object handles only one sample at a time.
    t = time.time()
    m32 = RunModel(samples=x_mixed, ntasks=1, model_script='python_model.py', 
                 model_object_name='det_rvs_par', model_dir='Python_Runs', vec=False)
    t_par_python = time.time() - t
    print("\nTime for parallel execution:")
    print(t_par_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m32.qoi_list)


Time for parallel execution:
0.009226083755493164

The values returned from the Matlab simulation:
[array([0.05088167]), array([-0.04821253]), array([-0.33834807]), array([-0.05412348]), array([0.36551101])]


## Example 4: Passing a fixed variable and an array of Random Variables to RunModel
In this example, we pass a fixed value coefficient as well as an array of random variables into a Python model.

Again, we will pass the samples both as an ndarray and as a list. Recall that UQpy converts all samples into an ndarray of at least two dimensions internally.

In [14]:
if pick_model == 'mixed' or pick_model == 'all':
    # Call MCS to generate samples
    x = 2.5
    
    print('Constant Coefficient:')
    print(x)
    print()
    
    print("Monte Carlo samples of a 3x3 matrix of standard normal random variables.")
    print('Samples stored as an array:')
    print('Data type:', type(x_mcs_array))
    print('Number of samples:', len(x_mcs_array))
    print('Dimensions of samples:', np.shape(x_mcs_array))
    print('Samples')
    print(x_mcs_array)
    print()
    
    x_mcs_list = list(x_mcs_array)

    print("3x3 matrix of standard normal random variables.")
    print('Samples stored as ndarray:')
    print('Data type:', type(x_mcs_list))
    print('Number of samples:', len(x_mcs_list))
    print('Dimensions of samples:', np.shape(x_mcs_list))
    print('Samples')
    print(x_mcs_list)
    print()
    
    # Notice that, in both the ndarray case and the list case, the samples have dimension (5,2). That is, there
    # are five samples of two variables. The first variable is a scalar. The second variable is a 3x3 matrix.

Constant Coefficient:
2.5

Monte Carlo samples of a 3x3 matrix of standard normal random variables.
Samples stored as an array:
Data type: <class 'numpy.ndarray'>
Number of samples: 5
Dimensions of samples: (5, 3, 3)
Samples
[[[ 2.1535102   0.1799892  -0.16889527]
  [-0.5214708  -1.22103583 -1.36417229]
  [ 1.95654882  1.58694939 -0.68619824]]

 [[-0.91615269 -0.17857194  0.03673987]
  [ 0.33356414 -0.67773133 -0.55117228]
  [ 0.59363143 -0.08381811 -1.23231183]]

 [[-0.03362657 -0.54695966 -0.00411439]
  [-0.29453249 -0.35960291  0.32253892]
  [ 0.48450073  0.78204757 -2.36459447]]

 [[-0.66210856  2.90532134 -1.56399743]
  [-0.35158539  0.26380749 -0.49864218]
  [-0.13391552 -0.1415161   0.34373841]]

 [[-1.59389912  1.09534476 -1.08910475]
  [ 0.57783749  0.21704457 -1.24612696]
  [ 0.64864639  0.306399   -0.21773583]]]

3x3 matrix of standard normal random variables.
Samples stored as ndarray:
Data type: <class 'list'>
Number of samples: 5
Dimensions of samples: (5, 3, 3)
Samples
[

### 4.1 Pass samples as ndarray, Python class called, serial execution 
This examples uses the following files:
- model_script = python_model.py

In [15]:
if pick_model == 'mixed' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m41 = RunModel(samples=x_mcs_array, ntasks=1, model_script='python_model.py', 
                 model_object_name='det_rvs_fixed', model_dir='Python_Runs', vec=False, coeff=x)
    t_ser_python = time.time() - t
    print("\nTime for serial execution:")
    print(t_ser_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m41.qoi_list)


Time for serial execution:
0.0158841609954834

The values returned from the Matlab simulation:
[14.144769325364933, -1.8101142834252881, 0.6889402518297437, 0.9968395370532651, -3.3007260477832103]


### 4.2 Pass samples as list, Python class called, serial execution 
This examples uses the following files:
- model_script = python_model.py

In [16]:
if pick_model == 'mixed' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m42 = RunModel(samples=x_mcs_list, ntasks=1, model_script='python_model.py', 
                 model_object_name='det_rvs_fixed', model_dir='Python_Runs', vec=False, coeff=x)
    t_ser_python = time.time() - t
    print("\nTime for serial execution:")
    print(t_ser_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m42.qoi_list)


Time for serial execution:
0.0066220760345458984

The values returned from the Matlab simulation:
[14.144769325364933, -1.8101142834252881, 0.6889402518297437, 0.9968395370532651, -3.3007260477832103]
